# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
weather_data = pd.read_csv("../WeatherPy/Output_Data/cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,thompson,55.74,-97.86,28.40,74,75,10.29,CA,1603325810
1,katsuura,35.13,140.30,64.40,63,75,9.17,JP,1603325661
2,arraial do cabo,-22.97,-42.03,73.13,88,9,14.41,BR,1603325721
3,lavrentiya,65.58,-171.00,38.17,85,0,5.77,RU,1603325814
4,provideniya,64.38,-173.30,41.00,80,90,2.24,RU,1603325755
...,...,...,...,...,...,...,...,...,...
576,kaduqli,11.02,29.72,77.68,67,96,4.47,SD,1603326563
577,atasu,48.68,71.64,41.76,66,100,16.40,KZ,1603326565
578,nago,26.59,127.98,77.00,94,90,6.93,JP,1603326566
579,rio gallegos,-51.62,-69.22,51.80,57,98,3.36,AR,1603326437


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Use the Lat and Lng as locations and Humitidy as the weight
locations = weather_data[["Lat", "Lng"]]

# Store Humidity
humidity = weather_data["Humidity"]

In [13]:
# Use the Lat and Lng as locations and Humitidy as the weight
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Narrow down that DataFrame to find the ideal weather condition (max temp lower than 80 but higher than 70, wind speed less than 10, 0 cloudiness)
narrowed_cities_df = weather_data.loc[(weather_data["Max Temp"] >= 70)& (weather_data["Max Temp"] <= 80) & (weather_data["Wind Speed"]<= 10) &(weather_data["Cloudiness"] ==0)].dropna()
narrowed_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
228,matay,28.42,30.78,71.37,67,0,6.80,EG,1603326106
232,morondava,-20.28,44.28,72.46,85,0,8.50,MG,1603325941
287,airai,-8.93,125.41,74.75,48,0,1.39,TL,1603325710
497,aswan,24.09,32.91,75.92,28,0,7.45,EG,1603326465
541,chongwe,-15.33,28.68,73.02,44,0,8.28,ZM,1603326520
565,ambad,19.62,75.80,73.94,87,0,3.87,IN,1603326550


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# Store variable named hotel_df
hotel_df = narrowed_cities_df.loc[:, ["City", "Country", "Lat", "Lng"]]

# Add "Hotel Name" to DataFrame
hotel_df["Hotel Name"] = ""

# Search for Hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel",
         "keyword": "hotel",
         "radius": 5000,
          "key": g_key}

In [16]:
# Creat the loop to find our hotels
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lng},{lat}"
    print({city_name})
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        time.sleep(1)
        
    except (KeyError, IndexError):
        print("Missing result")

{'matay'}
Missing result
{'morondava'}
Missing result
{'airai'}
Missing result
{'aswan'}
Missing result
{'chongwe'}
Missing result
{'ambad'}
Missing result


In [17]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
228,matay,EG,28.42,30.78,
232,morondava,MG,-20.28,44.28,
287,airai,TL,-8.93,125.41,
497,aswan,EG,24.09,32.91,
541,chongwe,ZM,-15.33,28.68,
565,ambad,IN,19.62,75.80,


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))